### load

In [1]:
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv('raw_data.csv')

# Print the first few rows of the data
data.head(10)


,pernr,cocode,payment_date,wage_type,wt_txt,number,amount,sbu,bus_area,job_id,job_txt,job_func_id,job_func_txt,job_fam_id,job_fam_txt,category
0,6000038,DURB,2020-04,2199,OT Adjustment,0.00,843.45,URC,DURB,81000535,Secretarial/Administrative Assistance,71000003.0,Administrative Services,91000008.0,General Administration,OT
1,6000041,DURB,2020-04,2199,OT Adjustment,0.00,958.47,URC,DURB,0,NaN,NaN,NaN,NaN,NaN,OT
2,6000047,DURB,2020-04,2199,OT Adjustment,0.00,1341.85,URC,DURB,0,NaN,NaN,NaN,NaN,NaN,OT
3,6000048,DURB,2020-04,2199,OT Adjustment,0.00,1915.34,URC,DURB,81001950,Supply Chain and Logistics Generalist/Mu,71000095.0,Supply Chain and Logistics,91000022.0,"Supply Chain, Logistics and Procurement",OT
4,9000004,NURC,2020-01,1026,Rest Day Pay,16.00,2685.92,URC,NUR,81001726,Production/Process Equipment Operations,71000056.0,Manufacturing/Operations,91000017.0,Manufacturing,Premium Pay
5,9000004,NURC,2020-01,1026,Rest Day Pay,16.00,2685.92,URC,NUR,81001726,Production/Process Equipment Operations,71000056.0,Manufacturing/Operations,91000017.0,Manufacturing,Premium Pay
6,9000004,NURC,2020-02,1026,Rest Day Pay,8.00,1342.96,URC,NUR,81001726,Production/Process Equipment Operations,71000056.0,Manufacturing/Operations,91000017.0,Manufacturing,Premium Pay
7,9000004,NURC,2020-02,1026,Rest Day Pay,6.77,1136.48,URC,NUR,81001726,Production/Process Equipment Operations,71000056.0,Manufacturing/Operations,91000017.0,Manufacturing,Premium Pay
8,9000004,NURC,2020-03,1026,Rest Day Pay,16.00,2685.92,URC,NUR,81001726,Production/Process Equipment Operations,71000056.0,Manufacturing/Operations,91000017.0,Manufacturing,Premium Pay
9,9000004,NURC,2020-04,1026,Rest Day Pay,8.00,1342.96,URC,NUR,81001726,Production/Process Equipment Operations,71000056.0,Manufacturing/Operations,91000017.0,Manufacturing,Premium Pay


### clean

In [2]:
# drop columns 'cocode', 'wage_type', 'sbu', 'job_id', 'job_func_id', 'job_func_txt', 'job_fam_id', 'job_fam_txt'	
data = data.drop(['cocode', 'wage_type', 'sbu', 'job_id', 'job_txt', 'job_func_id', 'job_func_txt', 'job_fam_id'], axis=1)

# replace the pernr with a randomized number with range '0001' to '9999'
import random
# all instances of the same pernr must be replaced with the same random number
pernr_unique = data['pernr'].unique()
pernr_random = random.sample(range(1, 9999), len(pernr_unique))
pernr_dict = dict(zip(pernr_unique, pernr_random))
data['pernr'] = data['pernr'].map(pernr_dict)

# replace column names: 'payment_data' -> 'month', 'wt_txt' -> 'overtime_type', 'number' -> 'hours', 'bus_area' -> 'location', 'job_txt' -> 'job_description'
data = data.rename(columns={'pernr': 'employee-id', 'payment_date': 'month', 'wt_txt': 'overtime-type', 'number': 'hours', 'bus_area': 'location', 'job_fam_txt': 'job-family', 'category': 'overtime-category'})

# drop rows where the value fo 'job_family' is missing or NaN/null
data = data.dropna(subset=['job-family'])

# show the first few rows of the data
data.head(10) 

,employee-id,month,overtime-type,hours,amount,location,job-family,overtime-category
0,6688,2020-04,OT Adjustment,0.00,843.45,DURB,General Administration,OT
3,8234,2020-04,OT Adjustment,0.00,1915.34,DURB,"Supply Chain, Logistics and Procurement",OT
4,4680,2020-01,Rest Day Pay,16.00,2685.92,NUR,Manufacturing,Premium Pay
5,4680,2020-01,Rest Day Pay,16.00,2685.92,NUR,Manufacturing,Premium Pay
6,4680,2020-02,Rest Day Pay,8.00,1342.96,NUR,Manufacturing,Premium Pay
7,4680,2020-02,Rest Day Pay,6.77,1136.48,NUR,Manufacturing,Premium Pay
8,4680,2020-03,Rest Day Pay,16.00,2685.92,NUR,Manufacturing,Premium Pay
9,4680,2020-04,Rest Day Pay,8.00,1342.96,NUR,Manufacturing,Premium Pay
10,4680,2020-04,Rest Day Pay,16.00,2685.92,NUR,Manufacturing,Premium Pay
11,4680,2020-05,Rest Day Pay,6.00,1064.64,NUR,Manufacturing,Premium Pay


In [3]:
# list unique values in column 'overtime_type'
data['overtime-type'].unique()

# drop rows where column value for 'overtime_type' are NOT any of the following: 'Rest Day Pay', 'Rest Day on Holiday Pay', 'Rest Day Premium Pay', 'Special Holiday Premium', 'Legal Holiday Premium Pay' 'OT Regular', 'OT Rest day', 'OT Special Holiday', 'OT Legal Holiday', 'Night Premium', 'Legal Holiday Pay'
data = data[data['overtime-type'].isin(['Rest Day Pay', 'Rest Day on Holiday Pay', 'Rest Day Premium Pay', 'Special Holiday Premium', 'Legal Holiday Premium Pay', 'OT Regular', 'OT Rest day', 'OT Special Holiday', 'OT Legal Holiday', 'Night Premium', 'Legal Holiday Pay'])]

# show random 10 rows
data.sample(10)


,employee-id,month,overtime-type,hours,amount,location,job-family,overtime-category
163512,2095,2020-12,Special Holiday Premium,2.00,41.62,BCFG,Manufacturing,Premium Pay
551218,8838,2021-05,OT Regular,37.15,2525.83,NUR,Manufacturing,OT
613715,5870,2021-12,Rest Day Premium Pay,8.00,201.04,BCFG,Manufacturing,Premium Pay
358865,3310,2020-06,OT Special Holiday,4.00,515.04,BCFG,Quality Management,OT
481523,930,2020-11,Rest Day Premium Pay,16.00,940.00,SURS,Legal and Compliance,Premium Pay
996868,865,2021-04,Night Premium,44.00,353.32,AIGC,Manufacturing,Night Differential
957914,2115,2021-12,Special Holiday Premium,8.00,151.92,AIGP,Manufacturing,Premium Pay
67721,1937,2020-09,OT Rest day,4.00,891.52,BCFG,Manufacturing,OT
929301,5,2021-05,Legal Holiday Premium Pay,8.00,434.88,BCFG,Engineering and Technical Support,Premium Pay
270631,7726,2020-03,OT Rest day,8.00,1005.36,SSON,Engineering and Technical Support,OT


In [4]:
# display unique values in column 'location'
data['location'].unique()

# replace column values in 'location' using the following mapping. 
# 'Factory 1' = ['AIGC', 'AIGP', 'AIGH', 'AIGR, 'BOPP', 'SSON', 'SHOF', 'CFC']
# 'Factory 2': ['SCAR', 'SPAS', 'SURS', 'SBAL', 'SDIS', 'CMCF', 'STOL']
# 'Factory 3': ['NUR', 'VURC', 'URCM', 'BCFG',]
# the 'Factory' are the replacement values for the 'location' values in the list

location_dict = {'AIGC': 'Factory 1', 'AIGP': 'Factory 1', 'AIGH': 'Factory 1', 'AIGR': 'Factory 1', 'BOPP': 'Factory 1', 'SSON': 'Factory 1', 'SHOF': 'Factory 1', 'CFC': 'Factory 1', 'SCAR': 'Factory 2', 'SPAS': 'Factory 2', 'SURS': 'Factory 2', 'SBAL': 'Factory 2', 'SDIS': 'Factory 2', 'CMCF': 'Factory 2', 'STOL': 'Factory 2', 'NUR': 'Factory 3', 'VURC': 'Factory 3', 'URCM': 'Factory 3', 'BCFG': 'Factory 3'}
data['location'] = data['location'].map(location_dict)

# show random 10 rows
data.head(10)

,employee-id,month,overtime-type,hours,amount,location,job-family,overtime-category
4,4680,2020-01,Rest Day Pay,16.00,2685.92,Factory 3,Manufacturing,Premium Pay
5,4680,2020-01,Rest Day Pay,16.00,2685.92,Factory 3,Manufacturing,Premium Pay
6,4680,2020-02,Rest Day Pay,8.00,1342.96,Factory 3,Manufacturing,Premium Pay
7,4680,2020-02,Rest Day Pay,6.77,1136.48,Factory 3,Manufacturing,Premium Pay
8,4680,2020-03,Rest Day Pay,16.00,2685.92,Factory 3,Manufacturing,Premium Pay
9,4680,2020-04,Rest Day Pay,8.00,1342.96,Factory 3,Manufacturing,Premium Pay
10,4680,2020-04,Rest Day Pay,16.00,2685.92,Factory 3,Manufacturing,Premium Pay
11,4680,2020-05,Rest Day Pay,6.00,1064.64,Factory 3,Manufacturing,Premium Pay
12,4680,2020-06,Rest Day Pay,16.00,2839.04,Factory 3,Manufacturing,Premium Pay
13,4680,2020-07,Rest Day Pay,8.00,1419.52,Factory 3,Manufacturing,Premium Pay


In [5]:
# show unique values in column 'month'
data['month'].unique()

# # display data type of value in column 'month'
# type(data['month'].iloc[0])

# drop rows where value of month is in this list: '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-09', '2021-10', '2021-11', '2021-12', '2021-07', '2021-08'
data = data[~data['month'].isin(['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-09', '2021-10', '2021-11', '2021-12', '2021-07', '2021-08'])]

# modify the values in 'month' column by replacing '2020' with '2015' in the string values: ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12']
data['month'] = data['month'].str.replace('2020', '2014')

# change the format of the values in 'month' column to 'month-year' from 'year-month'
data['month'] = pd.to_datetime(data['month'])
data['month'] = data['month'].dt.strftime('%m-%Y')

data = data.rename(columns={'month': 'month-year'})

# rename the values in column location using this mapping dict: {Factory 1: Innovatech Foundry, Factory 2: PrecisionWorks Plant, Factory 3: GreenForge Facility}
location_dict = {'Factory 1': 'Factory A', 'Factory 2': 'Factory B', 'Factory 3': 'Factory C'}
data['location'] = data['location'].map(location_dict)

# replace 01-2014 with January 2014, 02-2014 with February 2014, 03-2014 with March 2014, 04-2014 with April 2014, 05-2014 with May 2014, 06-2014 with June 2014, 07-2014 with July 2014, 08-2014 with August 2014, 09-2014 with September 2014, 10-2014 with October 2014, 11-2014 with November 2014, 12-2014 with December 2014
month_dict = {'01-2014': 'January 2014', '02-2014': 'February 2014', '03-2014': 'March 2014', '04-2014': 'April 2014', '05-2014': 'May 2014', '06-2014': 'June 2014', '07-2014': 'July 2014', '08-2014': 'August 2014', '09-2014': 'September 2014', '10-2014': 'October 2014', '11-2014': 'November 2014', '12-2014': 'December 2014'}
data['month-year'] = data['month-year'].map(month_dict)

data['month-year'].unique()

array(['January 2014', 'February 2014', 'March 2014', 'April 2014',
       'May 2014', 'June 2014', 'July 2014', 'August 2014',
       'September 2014', 'October 2014', 'November 2014', 'December 2014'],
      dtype=object)

In [6]:
# count the number of rows using filter 'location' == 'Factory A' and 'month-year' == 'January 2014'
data[(data['location'] == 'Factory A') & (data['month-year'] == 'January 2014')].count()

data[(data['location'] == 'Factory A') & (data['month-year'] == 'January 2014')].head(10)



,employee-id,month-year,overtime-type,hours,amount,location,job-family,overtime-category
26750,9420,January 2014,Special Holiday Premium,7.95,470.40,Factory A,"Supply Chain, Logistics and Procurement",Premium Pay
26764,9420,January 2014,OT Regular,70.76,17445.17,Factory A,"Supply Chain, Logistics and Procurement",OT
26765,9420,January 2014,OT Regular,64.65,15938.81,Factory A,"Supply Chain, Logistics and Procurement",OT
26807,9420,January 2014,Night Premium,8.00,157.76,Factory A,"Supply Chain, Logistics and Procurement",Night Differential
205467,5125,January 2014,OT Regular,9.50,1447.14,Factory A,Manufacturing,OT
205468,5125,January 2014,OT Regular,5.50,837.82,Factory A,Manufacturing,OT
205475,4616,January 2014,Rest Day Pay,16.00,2158.24,Factory A,Manufacturing,Premium Pay
205476,4616,January 2014,Rest Day Pay,8.00,1079.12,Factory A,Manufacturing,Premium Pay
205499,4616,January 2014,Rest Day Premium Pay,16.00,647.52,Factory A,Manufacturing,Premium Pay
205500,4616,January 2014,Rest Day Premium Pay,8.00,323.76,Factory A,Manufacturing,Premium Pay


In [7]:
# remove row index number
data = data.reset_index(drop=True)

# drop first row
data = data.drop([0])

data

,employee-id,month-year,overtime-type,hours,amount,location,job-family,overtime-category
1,4680,January 2014,Rest Day Pay,16.00,2685.92,Factory C,Manufacturing,Premium Pay
2,4680,February 2014,Rest Day Pay,8.00,1342.96,Factory C,Manufacturing,Premium Pay
3,4680,February 2014,Rest Day Pay,6.77,1136.48,Factory C,Manufacturing,Premium Pay
4,4680,March 2014,Rest Day Pay,16.00,2685.92,Factory C,Manufacturing,Premium Pay
5,4680,April 2014,Rest Day Pay,8.00,1342.96,Factory C,Manufacturing,Premium Pay
...,...,...,...,...,...,...,...,...
360733,1505,December 2014,OT Rest day,3.75,338.10,Factory C,Manufacturing,OT
360734,1505,December 2014,OT Rest day,4.00,360.64,Factory C,Manufacturing,OT
360735,1505,December 2014,OT Legal Holiday,4.00,554.84,Factory C,Manufacturing,OT
360736,1505,December 2014,Night Premium,8.00,42.72,Factory C,Manufacturing,Night Differential


In [8]:
# save df to csv
data.to_csv('final_data.csv', index=False)
data.to_csv('/home/stepanogil/ChatBotDev/hr-data-analysis-bot/final_data.csv', index=False)
